# Study Area

## 依赖包安装

运行本 notebook 需要以下 Python 包：

```bash
# 使用 pip 安装
pip install folium geopandas plotly

# 或使用 poetry（推荐）
poetry add folium geopandas plotly
```

主要包说明：
- **folium**: Leaflet.js 的 Python 封装，用于创建交互式地图
- **geopandas**: 处理地理空间数据，读取 shapefile
- **plotly**: 创建交互式图表和地图


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

import os
from hydra import compose, initialize

with initialize(version_base=None, config_path="../config"):
    cfg = compose(config_name="config.yaml")
os.chdir(cfg.ds.root)

## 方案一：使用 Folium (Leaflet) 绘制研究区

Folium 是 Leaflet.js 的 Python 封装，适合绘制交互式地图。

In [ ]:
import folium
import geopandas as gpd

# Read the shapefile
shp_path = Path(cfg.ds.yr_shp.folder).expanduser() / cfg.ds.yr_shp.outlines.huanghe
gdf = gpd.read_file(shp_path)

# Convert to projected CRS to calculate centroid, then back to WGS84
gdf_projected = gdf.to_crs("EPSG:3857")
center_lat = gdf_projected.geometry.centroid.to_crs("EPSG:4326").y.mean()
center_lon = gdf_projected.geometry.centroid.to_crs("EPSG:4326").x.mean()

# Create a folium map without default tiles
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=6,
    tiles=None,  # Don't add default tiles, we'll add them manually
)

# Add multiple base map layers
# 1. OpenStreetMap - 标准街景地图
folium.TileLayer(
    tiles="OpenStreetMap", name="街景地图 (OpenStreetMap)", overlay=False, control=True
).add_to(m)

# 2. Satellite imagery - 卫星影像 (Esri World Imagery)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="卫星影像 (Satellite)",
    overlay=False,
    control=True,
).add_to(m)

# 3. Terrain map - 地形图
folium.TileLayer(
    tiles="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
    attr="OpenTopoMap",
    name="地形图 (Terrain)",
    overlay=False,
    control=True,
).add_to(m)

# 4. CartoDB Positron - 简洁街景
folium.TileLayer(
    tiles="CartoDB positron", name="简洁街景 (CartoDB)", overlay=False, control=True
).add_to(m)

# 5. CartoDB Dark Matter - 深色主题
folium.TileLayer(
    tiles="CartoDB dark_matter", name="深色主题 (Dark)", overlay=False, control=True
).add_to(m)

# Add the Yellow River Basin boundary as an overlay
folium.GeoJson(
    gdf,
    name="黄河流域边界",
    style_function=lambda x: {
        "fillColor": "#3186cc",
        "color": "#0d47a1",
        "weight": 2,
        "fillOpacity": 0.2,
    },
    highlight_function=lambda x: {"weight": 3, "fillOpacity": 0.4},
    tooltip=folium.GeoJsonTooltip(
        fields=[],
        aliases=[],
        labels=True,
    ),
).add_to(m)

# Add layer control to switch between base maps
folium.LayerControl(position="topright").add_to(m)

# Display the map
m;

In [ ]:
m

## 世界对比


### Tab1: 水量对比

In [ ]:
# Import necessary libraries
import plotly.graph_objects as go


# Read data for discharge analysis
data = pd.read_excel("data/rivers.xlsx", sheet_name="Sheet1")
print("Data shape:", data.shape)
print("Columns:", data.columns.tolist())
data.head()


# Create Figure 1: Discharge bubble chart
x = data["basin"]  # Drainage basin area
y = data["River Length (km)"]  # River length
s = data["New_GRDC"]  # Discharge
rivers = data["River"]
per = data["WRperCaptia"]  # Water resources per capita

# Calculate discharge in 10^8 m^3/a
discharge_annual = s * 60 * 60 * 24 * 365 / 10**8

# Create custom hover text
hover_text = [
    f"<b>{river}</b><br>"
    + f"Basin area: {basin:.0f} km²<br>"
    + f"River length: {length:.0f} km<br>"
    + f"Discharge: {discharge:.1f}×10⁸ m³/a<br>"
    + f"Per capita: {pc:.0f} m³/a"
    for river, basin, length, discharge, pc in zip(rivers, x, y, discharge_annual, per)
]

# Create the bubble chart
fig1 = go.Figure()

# Add main scatter trace
fig1.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode="markers",
        marker=dict(
            size=s**0.5 / 3,  # Adjust size scaling for Plotly
            color=np.log10(per),  # Use log scale for color
            colorscale=[
                [0, "#E8C872"],
                [0.4, "#FFF3CF"],
                [0.6, "#C9D7DD"],
                [1, "#637A9F"],
            ],
            showscale=True,
            colorbar=dict(
                title=dict(text="Annual discharge<br>per capita (m³/a)", side="right"),
                tickmode="array",
                tickvals=[3, 4, 5],
                ticktext=["10³", "10⁴", "10⁵"],
                x=1.02,
            ),
            line=dict(width=0),
        ),
        text=hover_text,
        hovertemplate="%{text}<extra></extra>",
        name="Rivers",
    )
)

# (Interactive) remove static reference markers for discharge legend

# Highlight Yellow River
yellow_idx = rivers[rivers == "Yellow River"].index[0]
yellow_x = x[yellow_idx]
yellow_y = y[yellow_idx]

# Add vertical line for Yellow River
fig1.add_shape(
    type="line",
    x0=yellow_x,
    y0=0,
    x1=yellow_x,
    y1=yellow_y,
    line=dict(color="#8E2D30", width=1, dash="dash"),
)

# Add horizontal line for Yellow River
fig1.add_shape(
    type="line",
    x0=0,
    y0=yellow_y,
    x1=yellow_x,
    y1=yellow_y,
    line=dict(color="#8E2D30", width=1, dash="dash"),
)

# Add Yellow River annotation
fig1.add_annotation(
    x=750,
    y=6100,
    text="Yellow River",
    showarrow=False,
    font=dict(size=12, color="#8E2D30"),
)

# Add discharge value annotation for Yellow River
fig1.add_annotation(
    x=280,
    y=5750,
    text=f"Discharge =<br>{discharge_annual[yellow_idx]:.1f}×10⁸ m³/a",
    showarrow=False,
    font=dict(size=10, color="#8E2D30"),
    align="left",
)

# Update layout
fig1.update_layout(
    title=dict(text="River Discharge Analysis", x=0.5, xanchor="center"),
    xaxis=dict(
        title="Drainage basin area (km²)",
        range=[10, 6200],
        showgrid=True,
        gridcolor="rgba(187, 187, 187, 0.6)",
        gridwidth=0.75,
        griddash="dot",
    ),
    yaxis=dict(
        title="River length (km)",
        range=[1010, 6990],
        showgrid=True,
        gridcolor="rgba(187, 187, 187, 0.6)",
        gridwidth=0.75,
        griddash="dot",
    ),
    plot_bgcolor="white",
    width=900,
    height=600,
    font=dict(family="Arial, sans-serif", size=14),
    hovermode="closest",
)

fig1.show();

### Tab2: 输沙量对比

In [ ]:
# Read data for sediment load analysis
data2 = pd.read_excel("data/rivers.xlsx", sheet_name="Sheet2")
print("Data shape:", data2.shape)
data2.head()

# Create Figure 2: Sediment Load bubble chart
x2 = data2["basin"]  # Drainage basin area
y2 = data2["River Length (km)"]  # River length
s2 = data2["sediment_load"]  # Sediment load
rivers2 = data2["River"]
conc2 = data2["sediment_concentration"]  # Sediment concentration

# Create custom hover text
hover_text2 = [
    f"<b>{river}</b><br>"
    + f"Basin area: {basin:.0f} km²<br>"
    + f"River length: {length:.0f} km<br>"
    + f"Sediment load: {load:.2f}×10⁸ t/a<br>"
    + f"Concentration: {conc:.2f} kg·m⁻³"
    for river, basin, length, load, conc in zip(rivers2, x2, y2, s2, conc2)
]

# Create the bubble chart
fig2 = go.Figure()

# Add main scatter trace
fig2.add_trace(
    go.Scatter(
        x=x2,
        y=y2,
        mode="markers",
        marker=dict(
            size=s2**0.5 * 20,  # Adjust size scaling for Plotly
            color=np.log10(conc2),  # Use log scale for color
            colorscale=[
                [0, "#637A9F"],
                [0.4, "#C9D7DD"],
                [0.6, "#FFF3CF"],
                [1, "#E8C872"],
            ],
            showscale=True,
            colorbar=dict(
                title=dict(text="Sediment concentration<br>(kg·m⁻³)", side="right"),
                tickmode="array",
                tickvals=[-1, 0, 1],
                ticktext=["0.1", "1", "10"],
                x=1.02,
            ),
            line=dict(width=0.5, color="white"),
        ),
        text=hover_text2,
        hovertemplate="%{text}<extra></extra>",
        name="Rivers",
    )
)

# (Interactive) remove static reference markers for sediment load legend

# Highlight Yellow River
yellow_idx2 = rivers2[rivers2 == "Yellow River"].index[0]
yellow_x2 = x2[yellow_idx2]
yellow_y2 = y2[yellow_idx2]

# Add vertical line for Yellow River
fig2.add_shape(
    type="line",
    x0=yellow_x2,
    y0=0,
    x1=yellow_x2,
    y1=yellow_y2,
    line=dict(color="#8E2D30", width=1, dash="dash"),
)

# Add horizontal line for Yellow River
fig2.add_shape(
    type="line",
    x0=0,
    y0=yellow_y2,
    x1=yellow_x2,
    y1=yellow_y2,
    line=dict(color="#8E2D30", width=1, dash="dash"),
)

# Add Yellow River annotation
fig2.add_annotation(
    x=750,
    y=6100,
    text="Yellow River",
    showarrow=False,
    font=dict(size=12, color="#8E2D30"),
)

# Add sediment load value annotation for Yellow River
fig2.add_annotation(
    x=280,
    y=5750,
    text=f"Sediment load =<br>{s2[yellow_idx2]:.2f}×10⁸ t/a",
    showarrow=False,
    font=dict(size=10, color="#8E2D30"),
    align="left",
)

# Update layout
fig2.update_layout(
    title=dict(text="River Sediment Load Analysis", x=0.5, xanchor="center"),
    xaxis=dict(
        title="Drainage basin area (km²)",
        range=[10, 6200],
        showgrid=True,
        gridcolor="rgba(187, 187, 187, 0.6)",
        gridwidth=0.75,
        griddash="dot",
    ),
    yaxis=dict(
        title="River length (km)",
        range=[1010, 6990],
        showgrid=True,
        gridcolor="rgba(187, 187, 187, 0.6)",
        gridwidth=0.75,
        griddash="dot",
    ),
    plot_bgcolor="white",
    width=900,
    height=600,
    font=dict(family="Arial, sans-serif", size=14),
    hovermode="closest",
)

fig2.show();